<a href="https://colab.research.google.com/github/asmigemini/ocr/blob/main/ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from torch.utils.data import Dataset, DataLoader


In [ ]:
!apt-get install -y poppler-utils tesseract-ocr
!pip install pdf2image pytesseract pymupdf pillow opencv-python


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (199 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.7 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from google.colab import files
import cv2
import numpy as np


In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print("Uploaded:", pdf_path)


Saving asmi aadhaar.pdf to asmi aadhaar.pdf
Uploaded: asmi aadhaar.pdf


In [ ]:
def extract_text_digital(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()


In [ ]:
def preprocess_for_ocr(pil_img):
    img = np.array(pil_img.convert("L"))
    img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)[1]
    return img


In [ ]:
def extract_text_scanned(pdf_path):
    pages = convert_from_path(pdf_path, dpi=300)
    text = ""

    for page in pages:
        img = preprocess_for_ocr(page)
        page_text = pytesseract.image_to_string(img, lang="eng")
        text += page_text + "\n"

    return text.strip()


In [ ]:
def extract_all_text(pdf_path):
    digital_text = extract_text_digital(pdf_path)

    if len(digital_text) > 50:
        print("📄 Digital PDF detected — extracting embedded text")
        return digital_text
    else:
        print("📷 Scanned PDF detected — performing OCR")
        return extract_text_scanned(pdf_path)


In [ ]:
full_text = extract_all_text(pdf_path)

print("===== EXTRACTED TEXT =====")
print(full_text)  # preview


📄 Digital PDF detected — extracting embedded text
===== EXTRACTED TEXT =====
नामांकन क्रम/ Enrolment No.: 0807/80501/04176
To
अस्मी जेमिनी
Asmi Gemini
S-D-25,
TOWER APPARTMENT,
PITAM PURA DELHI,
VTC: Saraswati Vihar S.O,
District: North West Delhi,
State: Delhi,
PIN Code: 110034,
Mobile: 8447234237
7702 8659 7693
VID : 9122 5443 4721 2135
Details as on: 18/11/2025
Aadhaar no. issued: 28/08/2013
अस्मी जेमिनी
Asmi Gemini
जन्म तिथि/DOB: 07/12/2004
महिला/ FEMALE
7702 8659 7693
पता:
एस-डी0-25, टॉवर अपार्टमेंट, पीतम पुरा दिल्ली, सरस्वती
विहार एस.ओ, नोर्थ वेस्ट दिल्ली,
दिल्ली - 110034
Address:
S-D-25, TOWER APPARTMENT, PITAM PURA
DELHI, Saraswati Vihar S.O, DIST: North West
Delhi,
Delhi - 110034
7702 8659 7693
VID : 9122 5443 4721 2135


In [ ]:
from transformers import pipeline
import torch

corrector = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=-1  # CPU is fine
)

def correct_ocr(text):
    prompt = f"""
You are an OCR post-processing system.

Fix spelling, spacing, and punctuation errors.
Do NOT add new information.
Return ONLY the corrected text.

Example:
Input: intemet addre s iwww epa gov
Output: internet address www.epa.gov

Input: {text}
Output:
""".strip()

    result = corrector(
        prompt,
        do_sample=False
    )

    return result[0]["generated_text"].strip()
#test = "intemet addre s iwww epa gov"
#print("Before:", full_text[:2000])
print("After :", correct_ocr(full_text))
l=[]

for i in range(0,len(full_text),256):
    l.append(correct_ocr(full_text[i:i+256]))
l=" ".join(l)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


After : / Enrolment No.: 0807/80501/04176 To   Asmi Gemini S-D-25, TOWER APPARTMENT, PITAM PURA DELHI, VTC: Saraswati Vihar S.O, District: North West Delhi, State: Delhi, PIN Code: 110034, Mobile: 8447234237 7702 8659 7693 VID : 9122 5443 4721 2135 Details as on: 18/11/2025 Aadhaar no. issued: 28/08/2013   Asmi Gemini  /DOB: 07/12/2004 / FEMALE 7702 8659 7693 : -0-25,  ,   ,   ,   ,   ,   ,  - 110034 Address: S-D-25, TOWER APPARTMENT, PITAM PURA DELHI, Saraswat


In [ ]:
def find_candidate_lines(text, user_field, intent):
    keywords = user_field.lower().split()+[i.lower() for i in intent]
    lines = text.split(". ")

    candidates = []
    for line in lines:
        if any(k in line.lower() for k in keywords):
            candidates.append(line)

    return candidates


In [ ]:
def correct_line_with_llm(line):
    prompt = f"""
      You are an OCR post-processing system.

      Fix spelling, spacing, and punctuation errors.
      Do NOT add new information.
      Return ONLY the corrected text.

      Example:
      Input: intemet addre s htIp:/ iwww epa gov
      Output: internet address http://www.epa.gov

      Input: {line}
      Output:
      """.strip()


    out = corrector(prompt, do_sample=False)
    return out[0]["generated_text"].strip()


In [ ]:
import regex as re
def extract_value(text, intent):

    if intent == "url":
        m = re.search(r'https?://?\S+|www\.\S+.\S+', text)
        return m.group(0) if m else None

    if intent == "email":
        m = re.search(r'\S+@\S+', text)
        return m.group(0) if m else None

    if intent == "gst":
        m = re.search(r'\b\d{2}[A-Z]{5}\d{4}[A-Z][A-Z\d]Z[A-Z\d]\b', text)
        return m.group(0) if m else None

    if intent == "phone":
        m = re.search(r'\b\d{10}\b', text)
        return m.group(0) if m else None

    if intent == "date":
        m = re.search(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', text)
        return m.group(0) if m else None

    if intent == "amount":
        m = re.search(r'₹?\s?\d+(?:,\d{3})*(?:\.\d{2})?', text)
        return m.group(0) if m else None

    return None



In [ ]:
def guess(line,user_field):

  line=line.split()
  user_field=user_field.split()
  for i in range(len(line)):
    for j in range(len(user_field)):
      if line[i].lower()==user_field[j].lower() or line[i].lower()[0:len(line[i])-1]==user_field[j].lower():
        return line[i+len(user_field)]
  return None


In [ ]:
def detect_intent(user_query):
    q = user_query.lower()

    if any(k in q for k in ["url", "website", "link", "internet"]):
        return "url", ["url", "website", "link", "internet"]

    if any(k in q for k in ["email", "mail"]):
        return "email", ["email", "mail"]

    if any(k in q for k in ["gst", "tax"]):
        return "gst", ["gst", "tax"]

    if any(k in q for k in ["phone", "mobile", "contact"]):
        return "phone", ["phone", "mobile", "contact"]

    if any(k in q for k in ["date"]):
        return "date", ["date"]

    if any(k in q for k in ["amount", "total", "price", "rs", "₹"]):
        return "amount", ["amount", "total", "price", "rs", "₹"]

    return "unknown", [] # Modified to return two values

In [ ]:
def document_ai(user_field):
    raw_text=l
    intent,l1 = detect_intent(user_field)
    candidates = find_candidate_lines(raw_text, user_field, l1)
    if not candidates:
        return {"status": "not found"}

    corrected = [correct_line_with_llm(c) for c in candidates]

    for line in corrected:
        value = extract_value(line, intent)
        if value:
            return {
                "requested_field": user_field,
                "value": value,
                "method": "ocr + ml + rules",
                "status": "success"
            }
        else:
          return {
                "requested_field": user_field,
                "value": guess(line, user_field),
                "method": "ocr + ml + rules",
                "status": "success"
            }


    return {"status": "value not found"}


In [ ]:
result = document_ai(
    user_field=input("enter key: ")
)

print(result)


enter key: state
{'requested_field': 'state', 'value': 'Delhi,', 'method': 'ocr + ml + rules', 'status': 'success'}
